In [1]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "MetaGraphs",
    "Graphs",
    "JSON",
    "uCSV",
    "DataFrames",
    "Dates",
    "Primes",
    "Kmers",
    "BioSequences",
    "FASTX",
    "ProgressMeter",
    "Random"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

# Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
   Installed RecipesPipeline ─ v0.6.6
  No Changes to `~/work/Mycelia/Project.toml`
    Updating `~/work/Mycelia/Manifest.toml`
  [01d81517] ↑ RecipesPipeline v0.6.5 ⇒ v0.6.6
Precompiling project...
  ✓ RecipesPipeline
  ✓ Plots
  ✓ StatsPlots
  ✓ Mycelia
  4 dependencies successfully precompiled in 79 seconds (234 already precompiled)
   Resolving package versions...
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`


In [2]:
#papermill parameters
base_directory = "$(homedir())/workspace/sars-cov2-pangenome-analysis/"
sequences_directory = "$(base_directory)/data/sequences"
metadata_file = "$(base_directory)/metadata/sequences.csv"
working_directory = "$(homedir())/workspace/scratch/$(Dates.today())"

"/home/jovyan/workspace/scratch/2022-10-01"

In [3]:
neo4j_username = "neo4j"

# remote_neo4j_address = ENV["NEO4J_URL"]
# remote_neo4j_password = ENV["NEO4J_PASSWORD"]
local_neo4j_bolt_address = "bolt://localhost:7687"
local_neo4j_http_address = "neo4j://localhost:7474"
# local_neo4j_password = "neo4j"
# local_neo4j_password = Random.randstring(7)
local_neo4j_password = "ii0sRIc"

neo4j_local_import_directory = "/home/jovyan/.local/neo4j-community-4.4.11/import"

"/home/jovyan/.local/neo4j-community-4.4.11/import"

In [36]:
NEO4J_PATH="/home/jovyan/.local/neo4j-community-4.4.11"

"/home/jovyan/.local/neo4j-community-4.4.11"

In [23]:
run(`$(NEO4J_PATH)/bin/neo4j stop`)
run(`$(NEO4J_PATH)/bin/neo4j start`)
run(`$(NEO4J_PATH)/bin/neo4j status`)

Neo4j is not running.
Directories in use:
home:         /home/jovyan/.local/neo4j-community-4.4.11
config:       /home/jovyan/.local/neo4j-community-4.4.11/conf
logs:         /home/jovyan/.local/neo4j-community-4.4.11/logs
plugins:      /home/jovyan/.local/neo4j-community-4.4.11/plugins
import:       /home/jovyan/.local/neo4j-community-4.4.11/import
data:         /home/jovyan/.local/neo4j-community-4.4.11/data
certificates: /home/jovyan/.local/neo4j-community-4.4.11/certificates
licenses:     /home/jovyan/.local/neo4j-community-4.4.11/licenses
run:          /home/jovyan/.local/neo4j-community-4.4.11/run
Starting Neo4j.
Started neo4j (pid:40596). It is available at http://localhost:7474
There may be a short delay until the server is ready.
Neo4j is running at pid 40596


Process(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j status`, ProcessExited(0))

In [4]:
# # # update password - only need to do this on initial setup
# cmd = "ALTER CURRENT USER SET PASSWORD FROM 'neo4j' TO '$(local_neo4j_password)'"
# run(Mycelia.cypher(cmd, address=local_neo4j_bolt_address, password="neo4j", username="neo4j", database="system"))

In [5]:
mkpath(working_directory)
cd(working_directory)

In [6]:
# f = metadata_file
n = 10^1
# n = 10^2
# n = 10^3
# n = 10^4
# n = 10^5
# n = 10^6
# n = countlines(metadata_file)
f = open(`head -n $(n) $metadata_file`)
@time sequence_metadata = DataFrames.DataFrame(uCSV.read(f, quotes='"', header=1, typedetectrows=100)...)

  3.891805 seconds (8.09 M allocations: 490.847 MiB, 3.75% gc time, 77.00% compilation time)


,Accession,SRA_Accession,Submitters
,String,String,String
1,NC_045512.2,,"Wu,F., Zhao,S., Yu,B., Chen,Y.M., Wang,W., Song,Z.G., Hu,Y., Tao,Z.W., Tian,J.H., Pei,Y.Y., Yuan,M.L., Zhang,Y.L., Dai,F.H., Liu,Y., Wang,Q.M., Zheng,J.J., Xu,L., Holmes,E.C., Zhang,Y.Z., Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F., Atkins,J.F., Howard,M.T., Robertson,M.P., Igel,H., Baertsch,R., Haussler,D., Ares,M. Jr., Scott,W.G., Williams,G.D., Chang,R.Y., Brian,D.A., Chen,Y.-M., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Zhang,Y.-L., Dai,F.-H., Wang,Q.-M., Zheng,J.-J., Zhang,Y.-Z."
2,OP164776.1,,"Afrad Moon,M.H., Qadri,F."
3,OP164777.1,,"Afrad Moon,M.H., Qadri,F."
4,OP164778.1,,"Afrad Moon,M.H., Qadri,F."
5,OP164779.1,,"Afrad Moon,M.H., Qadri,F."
6,OP164780.1,,"Afrad Moon,M.H., Qadri,F."
7,OP164781.1,,"Afrad Moon,M.H., Qadri,F."
8,OP164782.1,,"Afrad Moon,M.H., Qadri,F."
9,OP164783.1,,"Afrad Moon,M.H., Qadri,F."


In [7]:
fastx_files = map(x -> "$(sequences_directory)/$(x).fna", sequence_metadata[!, "Accession"])

9-element Vector{String}:
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/NC_045512.2.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164776.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164777.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164778.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164779.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164780.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164781.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164782.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164783.1.fna"

In [8]:
graph = Mycelia.initialize_graph()
graph = Mycelia.add_fastx_to_graph!(graph, fastx_files)
graph = Mycelia.add_metadata_from_table!(graph, sequence_metadata, identifier_column = "Accession")

{9, 0} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [9]:
# max_k = Mycelia.assess_dnamer_saturation(fastx_files)
# max_k = 31
# min_k=17
# max_k=17
min_k=max_k=31
kmer_sizes = Primes.primes(min_k, max_k)

1-element Vector{Int64}:
 31

In [10]:
for kmer_size in kmer_sizes
    @show kmer_size
    @time Mycelia.add_fasta_record_kmers_to_graph!(graph, kmer_size)
end

kmer_size = 31
  6.182926 seconds (22.09 M allocations: 1.759 GiB, 17.67% gc time)


In [11]:
for kmer_size in kmer_sizes
    @show kmer_size
    @time Mycelia.graph_to_gfa(graph, kmer_size)
end

kmer_size = 31
  1.210143 seconds (3.24 M allocations: 179.708 MiB, 14.70% gc time, 49.28% compilation time)


In [12]:
for gfa in filter(x -> occursin(r"\.gfa$", x), readdir(working_directory, join=true))
    @show gfa
    for nodewidth in [100, 500, 1000]
        gfa_img = "$(gfa).$(nodewidth).jpg"
        if !isfile(gfa_img)
            run(`Bandage image $gfa $gfa_img --deppower 1 --depwidth 1 --nodewidth $(nodewidth)`)
        end
    end
end

gfa = "/home/jovyan/workspace/scratch/2022-10-01/31.gfa"


In [13]:
run(`$(NEO4J_PATH)/bin/neo4j stop`)
run(`$(NEO4J_PATH)/bin/neo4j start`)
run(`$(NEO4J_PATH)/bin/neo4j status`)

Neo4j is not running.
Directories in use:
home:         /home/jovyan/.local/neo4j-community-4.4.11
config:       /home/jovyan/.local/neo4j-community-4.4.11/conf
logs:         /home/jovyan/.local/neo4j-community-4.4.11/logs
plugins:      /home/jovyan/.local/neo4j-community-4.4.11/plugins
import:       /home/jovyan/.local/neo4j-community-4.4.11/import
data:         /home/jovyan/.local/neo4j-community-4.4.11/data
certificates: /home/jovyan/.local/neo4j-community-4.4.11/certificates
licenses:     /home/jovyan/.local/neo4j-community-4.4.11/licenses
run:          /home/jovyan/.local/neo4j-community-4.4.11/run
Starting Neo4j.
Started neo4j (pid:22040). It is available at http://localhost:7474
There may be a short delay until the server is ready.
Neo4j is running at pid 22040


Process(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j status`, ProcessExited(0))

In [14]:
# Mycelia.list_databases(address=local_neo4j_bolt_address, password=local_neo4j_password)

In [15]:
Mycelia.create_node_constraints(graph, address=local_neo4j_bolt_address, password=local_neo4j_password)

cypher = `/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'CREATE CONSTRAINT ON (t:\`FASTX.FASTA.Record\`) ASSERT t.identifier IS UNIQUE'`
cypher = `/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'CREATE CONSTRAINT ON (t:\`Kmers.DNAKmer{31}\`) ASSERT t.identifier IS UNIQUE'`


Unable to connect to localhost:7687, ensure the database is running and that there is a working network connection to it.
An equivalent constraint already exists, 'Constraint( id=6, name='constraint_e07b72f7', type='UNIQUENESS', schema=(:Kmers.DNAKmer{31} {identifier}), ownedIndex=5 )'.


In [16]:
run(Mycelia.cypher("MATCH (n) DETACH DELETE n", address=local_neo4j_bolt_address, password=local_neo4j_password))
run(Mycelia.cypher("MATCH (n) RETURN count(n) as count", address=local_neo4j_bolt_address, password=local_neo4j_password))
# run(Mycelia.cypher("MATCH (n) DETACH DELETE n", address=local_neo4j_bolt_address, password=local_neo4j_password))

count
0


Process(`/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'MATCH (n) RETURN count(n) as count'`, ProcessExited(0))

In [17]:
Mycelia.upload_nodes_to_neo4j(graph=graph, address=local_neo4j_bolt_address, password=local_neo4j_password, neo4j_import_directory=neo4j_local_import_directory)

┌ Info: uploading node_type => FASTX.FASTA.Record...
└ @ Mycelia /home/jovyan/work/Mycelia/src/database-interaction.jl:79
┌ Info: uploading node_type => Kmers.DNAKmer{31}...
└ @ Mycelia /home/jovyan/work/Mycelia/src/database-interaction.jl:79
Progress: 100%|█████████████████████████████████████████| Time: 0:00:33
┌ Info: done!
└ @ Mycelia /home/jovyan/work/Mycelia/src/database-interaction.jl:88


In [18]:
run(Mycelia.cypher("MATCH (n) RETURN count(n) as count", address=local_neo4j_bolt_address, password=local_neo4j_password))

count
32147


Process(`/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'MATCH (n) RETURN count(n) as count'`, ProcessExited(0))

In [ ]:
# edge_types = unique(MetaGraphs.props(graph, e)[:TYPE] for e in Graphs.edges(graph))
# edge_type_strings = Mycelia.type_to_string.(edge_types)

In [24]:
function upload_edges_to_neo4j(;graph, address, username="neo4j", password, format="auto", database="neo4j", neo4j_import_directory)
    
    edge_types = unique(MetaGraphs.props(graph, e)[:TYPE] for e in Graphs.edges(graph))
    
    # FIX ME!!
    # for edge_type in edge_types
    for edge_type in edge_types[2:2]
        @info "uploading edge_type => $(Mycelia.type_to_string(edge_type))..."
        edge_type_table = edge_type_to_dataframe(edge_type=edge_type, graph=graph)
        try
            upload_edge_table(table=edge_type_table, address=address, password=password, neo4j_import_dir=neo4j_import_directory)
        catch e
            showerror(stdout, e)
        end
    end
    
    @info "done!"
end

function edge_type_to_dataframe(;edge_type, graph)
    edge_type_indices = filter(e -> MetaGraphs.props(graph, e)[:TYPE] == edge_type, collect(Graphs.edges(graph)))
    # @show edge_type_indices
    edge_type_parameters = unique(reduce(vcat, map(e -> collect(keys(MetaGraphs.props(graph, e))), edge_type_indices)))
    edge_type_table = DataFrames.DataFrame(Dict(p => [] for p in edge_type_parameters))
    for edge_index in edge_type_indices
        push!(edge_type_table, MetaGraphs.props(graph, edge_index))
    end
    return edge_type_table
end

function upload_edge_table(;table, window_size=1000, address, password, username="neo4j", database="neo4j", neo4j_import_dir)
    nrows = DataFrames.nrow(table)
    windows = (i:min(i+window_size-1,nrows) for i in 1:window_size:nrows)
    
    edge_types = unique(table[!, "TYPE"])
    @assert length(edge_types) == 1
    EDGE_TYPE = Mycelia.type_to_string(first(edge_types))
    parameters = ["$(n): row.$(n)" for n in filter(x -> !(x in ["TYPE"]), names(table))]
    parameters = "{" * join(parameters, ", ") * "}"

    ProgressMeter.@showprogress for (i, window) in enumerate(windows)
        df_sub = table[window, :]
        f = "edge$i.tsv"
        local_f_path = "$(neo4j_import_dir)/$(f)"
        uCSV.write(local_f_path, df_sub, delim='\t')
        run(`chmod 777 $(local_f_path)`)
        f_url = "file:///$(f)"
        cmd =
        """
        LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
        MATCH (src {identifier: row.src})
        MATCH (dst {identifier: row.dst})
        MERGE (src)-[p:`$(EDGE_TYPE)` $(parameters)]->(dst)
        """
        # had been merge above!
        # create Progress:   2%|▊                                        |  ETA: 1:04:39
        # merge Progress:   3%|█▍                                       |  ETA: 1:25:12
        # Progress:  12%|█████                                    |  ETA: 0:07:11
        cmd = rstrip(replace(cmd, '\n' => ' '))
        cypher_cmd = Mycelia.cypher(cmd, address = address, username = username, password = password, database = database)
        run(cypher_cmd) 
    end
end

upload_edge_table (generic function with 1 method)

In [25]:
upload_edges_to_neo4j(graph=graph, address=local_neo4j_bolt_address, password=local_neo4j_password, neo4j_import_directory=neo4j_local_import_directory)

┌ Info: uploading edge_type => Kmers.DNAKmer{32}...
└ @ Main In[24]:8
Progress: 100%|█████████████████████████████████████████| Time: 0:07:37
┌ Info: done!
└ @ Main In[24]:17


In [39]:
# datetime = replace(string(Dates.now()), r"[\-:.a-zA-Z]" => "")
neo4j_dump = "$(working_directory)/neo4j-covid-database-$(n)-$(datetime).dump"

"/home/jovyan/workspace/scratch/2022-10-01/neo4j-covid-database-10-20221001235948609.dump"

In [58]:
println("$(NEO4J_PATH)/bin/neo4j stop")

/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j stop


In [38]:
run(`$(NEO4J_PATH)/bin/neo4j stop`)
run(`$(NEO4J_PATH)/bin/neo4j-admin dump --database=neo4j --to=$(neo4j_dump)`)

Stopping Neo4j....... stopped.


Selecting JVM - Version:11.0.11+9-Ubuntu-0ubuntu2.20.04, Name:OpenJDK 64-Bit Server VM, Vendor:Ubuntu

Files: 1/38, data:  0.0%
Files: 1/38, data:  0.0%
Files: 2/38, data:  0.0%
Files: 3/38, data:  0.0%
Files: 4/38, data:  0.0%
Files: 5/38, data:  0.0%
Files: 6/38, data:  0.1%
Files: 7/38, data:  0.1%
Files: 8/38, data:  0.1%
Files: 9/38, data:  0.1%
Files: 10/38, data:  0.9%
Files: 11/38, data:  0.9%
Files: 12/38, data:  0.9%
Files: 13/38, data:  0.9%
Files: 14/38, data:  0.9%
Files: 15/38, data:  3.9%
Files: 16/38, data:  4.0%
Files: 17/38, data:  4.0%
Files: 18/38, data:  4.0%
Files: 19/38, data:  4.0%
Files: 20/38, data:  4.0%
Files: 21/38, data:  4.0%
Files: 22/38, data:  4.0%
Files: 23/38, data:  4.0%
Files: 24/38, data:  4.3%
Files: 25/38, data:  4.4%
Files: 26/38, data:  4.4%
Files: 27/38, data:  4.4%
Files: 28/38, data:  4.4%
Files: 29/38, data:  4.5%
Files: 30/38, data:  4.5%
Files: 31/38, data:  5.9%
Files: 32/38, data:  6.0%
Files: 33/38, data:  6.0%
Files: 34/38, data:  6.

Process(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j-admin dump --database=neo4j --to=/home/jovyan/workspace/scratch/2022-10-01/neo4j-covid-database-10-20221001235948609.dump`, ProcessExited(0))

In [42]:
# bin/neo4j-admin load --from=/dumps/neo4j/neo4j-<timestamp>.dump --database=neo4j --force

# ssh cameron@odin.lcfta.com

println("run me in the shell")
println("scp $(neo4j_dump) cameron@odin.lcfta.com:/mnt/coldstorage/neo4j/import/$(basename(neo4j_dump))")

run me in the shell
scp /home/jovyan/workspace/scratch/2022-10-01/neo4j-covid-database-10-20221001235948609.dump cameron@odin.lcfta.com:/mnt/coldstorage/neo4j/import/neo4j-covid-database-10-20221001235948609.dump


In [43]:
println("ssh cameron@odin.lcfta.com")

ssh cameron@odin.lcfta.com


In [ ]:
# println("sudo docker exec -it neo4j bash")

In [49]:
println("sudo docker exec -it neo4j /var/lib/neo4j/bin/neo4j-admin load --from=/var/lib/neo4j/import/$(basename(neo4j_dump)) --database=neo4j --force")

sudo docker exec -it neo4j /var/lib/neo4j/bin/neo4j-admin load --from=/var/lib/neo4j/import/neo4j-covid-database-10-20221001235948609.dump --database=neo4j --force


In [55]:
println("/var/lib/neo4j/bin/neo4j stop")

/var/lib/neo4j/bin/neo4j stop


In [57]:
println("/var/lib/neo4j/bin/neo4j-admin load --from=/var/lib/neo4j/import/$(basename(neo4j_dump)) --database=neo4j --force")

/var/lib/neo4j/bin/neo4j-admin load --from=/var/lib/neo4j/import/neo4j-covid-database-10-20221001235948609.dump --database=neo4j --force


In [61]:
println("$(NEO4J_PATH)/bin/neo4j-admin load --from=$(neo4j_dump) --database=neo4j --force")

/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j-admin load --from=/home/jovyan/workspace/scratch/2022-10-01/neo4j-covid-database-10-20221001235948609.dump --database=neo4j --force


In [56]:
# get container ID
# sudo docker container ls

# can call by the name - you can't

# println("sudo docker exec --interactive --tty 6844ac0dcadf neo4j-admin load --from=/var/lib/neo4j/import/neo4j-covid-database-10-20221001235948609.dump --database=neo4j --force")
println("sudo docker exec --interactive --tty neo4j bash")

sudo docker exec --interactive --tty 6844ac0dcadf bash


LoadError: UndefVarError: neo4j not defined

In [ ]:
# [cameron@hermod:/mnt/coldstorage/git/docker-neo4j]$ sudo docker rm neo4jloader
# neo4jloader

# [cameron@hermod:/mnt/coldstorage/git/docker-neo4j]$ sudo ./loaddb.sh 
# Selecting JVM - Version:11.0.16.1+1, Name:OpenJDK 64-Bit Server VM, Vendor:Eclipse Adoptium
# Done: 38 files, 266.0MiB processed.

# -d


-d

# sudo docker restart neo4j

In [ ]:
# sudo docker stop neo4j
# /mnt/coldstorage/git/docker-neo4j]